In [13]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import string
import nltk
nltk.download('stopwords')
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import preprocessing
# from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

[nltk_data] Error loading stopwords: <urlopen error [Errno -2] Name or
[nltk_data]     service not known>


In [14]:
#read the data set
data = pd.read_csv("./Data/multilabel_emotion_dataset.csv",sep="\t")
print data.shape
# data = pd.read_csv("./Data/phrases.csv")
# data=pd.DataFrame(data)
# print type(data)
# print data.shape
# data=data.dropna()
data_phrase=pd.DataFrame()
data_phrase['Phrase']=data['Tweet']
print data_phrase.head()
# print type(data_phrase)
df_emo=data
df_emo=df_emo.drop(["ID","Tweet"],axis=1)
data_emotion=df_emo.values
print data_emotion.shape

# print data_phrase
# print data['Emotion'].unique()
# print data.shape

(6839, 13)
                                              Phrase
0  [Worrying is a down payment on a problem you m...
1  Whatever you decide to do make sure it makes y...
2  @Max_Kellerman  it also helps that the majorit...
3                                  George S. Patton 
4  My roommate: it's okay that we can't spell bec...
(6839, 11)


In [15]:
print data_emotion

[[0 1 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]]


## Preprocess the data set via cleaning, tokenisation and lemmatization

In [17]:
# print data
import re

def clean_dataset(data_cf):
    translator = string.maketrans('', '')
    l=[]
    for index,row in data_cf.iterrows():
        row['Phrase'] = row['Phrase'].replace('[','')
        row['Phrase'] = row['Phrase'].replace(']','')
        row['Phrase'] = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL', row['Phrase'])
        row['Phrase'] = re.sub('@[^\s]+','USER', row['Phrase'])
#         row['Phrase'] = row['Phrase'].lower().replace("ё", "е")
        row['Phrase'] = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', row['Phrase'])
        row['Phrase'] = re.sub(' +',' ', row['Phrase'])
        row['Phrase'] = row['Phrase'].strip()
        row['Phrase']=re.sub(r'([^\s\w]|_)+', '', row['Phrase'])
        l.append(row['Phrase'])
#         print row['Phrase']
        
#         row['Phrase'] = row['Phrase'].translate(translator,string.punctuation)
    data_cf['Phrase']=l
    return data_cf
data_clean = clean_dataset(data_phrase)
print data_clean.head()

                                              Phrase
0  Worrying is a down payment on a problem you ma...
1  Whatever you decide to do make sure it makes y...
2  USER it also helps that the majority of NFL co...
3                                    George S Patton
4  My roommate it s okay that we can t spell beca...


In [18]:
def tokenise(data):
    ## Convert words to lower case and split them    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    l=[]
    for index,row in data.iterrows():
        text = str(row['Phrase']).lower().split(' ')
        text = [w.strip() for w in text if not w in stops and len(w) >= 2]
        text = " ".join(text)
        row['Phrase'] = text
        l.append(row['Phrase'])
    # split the dataset into tokens
    data['Phrase']=l
    return data
data_token = tokenise(data_clean)
data_token.head()

,Phrase
0,worrying payment problem may never joyce meyer...
1,whatever decide make sure makes happy
2,user also helps majority nfl coaching inept bi...
3,george patton
4,roommate okay spell autocorrect terrible first...


In [19]:
def lemmatization(dataset):
    stemmer = SnowballStemmer('english')
    l=[]
    for index,row in dataset.iterrows():
        text = str(row['Phrase']).split()
        stemmed_words = [stemmer.stem(word) for word in text]
#         list_of_words.append(stemmed_words)
        text = " ".join(stemmed_words)
        row['Phrase'] = text
        l.append(row['Phrase'])
        
    data['Phrase'] = l
    return dataset
data_stemmed = lemmatization(data_token)
data_stemmed.head()

,Phrase
0,worri payment problem may never joyc meyer mot...
1,whatev decid make sure make happi
2,user also help major nfl coach inept bill brie...
3,georg patton
4,roommat okay spell autocorrect terribl firstwo...


In [20]:
def bag_of_words(data):
    count_vect=CountVectorizer(encoding='latin-1')
    count_vect.fit(data) #creates vocab of words
#     print (count_vect.vocabulary_)
    data_phrase=count_vect.transform(data)
#     print(data_phrase.shape)
#     print(type(data_phrase))
#     print(data_phrase.toarray())
    return data_phrase,count_vect
data_phrase,count_vect=bag_of_words(data['Phrase'])
data_phrase.shape


(6839, 9611)

In [21]:
def tf_idf(data_phrase):
    tfidf_transformer = TfidfTransformer()
    phrase_tfidf = tfidf_transformer.fit_transform(data_phrase)
    return phrase_tfidf.toarray(),tfidf_transformer
phrase_tfidf,tfidf_transformer=tf_idf(data_phrase)
phrase_tfidf.shape

(6839, 9611)

In [52]:
# Import LabelEncoder
# #creating labelEncoder
# le = preprocessing.LabelEncoder()
# # Converting string labels into numbers.
# emotion_data=data['Emotion'].values
# emo_enc=le.fit_transform(emotion_data)
# print emo_enc



In [22]:
train_phrase, validate_phrase = np.split(phrase_tfidf,[int(.8*len(phrase_tfidf))])
# train_label,validate_label=np.split(emotion_data,[int(.8*len(emotion_data))])
train_label,validate_label=np.split(data_emotion,[int(.8*len(data_emotion))])
print "data",train_phrase.shape,validate_phrase.shape
print "label",train_label.shape,validate_label.shape

data (5471, 9611) (1368, 9611)
label (5471, 11) (1368, 11)


In [23]:
modelknn = KNeighborsClassifier(n_neighbors=3)
modelknn.fit(train_phrase, train_label)
label_predicted = modelknn.predict(validate_phrase)
 
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

sklearn_score_train = modelknn.score(train_phrase,train_label)
print("Sklearn's score on training data :",sklearn_score_train)
sklearn_score_test = modelknn.score(validate_phrase,validate_label)
print("Sklearn's score on testing data :",sklearn_score_test)
print("Classification report for testing data :-")
print(classification_report(validate_label, label_predicted))

("Sklearn's score on training data :", 0.3555108755254981)
("Sklearn's score on testing data :", 0.12938596491228072)
Classification report for testing data :-
              precision    recall  f1-score   support

           0       0.65      0.55      0.59       494
           1       0.34      0.10      0.15       209
           2       0.61      0.52      0.56       524
           3       0.61      0.40      0.49       272
           4       0.67      0.54      0.60       492
           5       0.38      0.26      0.31       134
           6       0.58      0.40      0.47       421
           7       0.24      0.13      0.17       160
           8       0.50      0.40      0.45       394
           9       0.44      0.16      0.23        69
          10       0.13      0.03      0.05        63

   micro avg       0.57      0.41      0.48      3232
   macro avg       0.47      0.32      0.37      3232
weighted avg       0.55      0.41      0.47      3232
 samples avg       0.52     

/home/himani/.local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/himani/.local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [25]:
mapping_dict={0:"anger",1:"anticipation",2:"disgust",3:"fear",4:"joy",5:"love",6:"optimism",7:"pessimism",8:"sadness",9:"surprise",10:"trust"}

In [30]:
test_sentence=raw_input("Enter text:")
test_sen=[]
test_sen.append(test_sentence)
bow=count_vect.transform(test_sen)
tf_test=tfidf_transformer.transform(bow).toarray()
predicted_test= modelknn.predict(tf_test)
# le.inverse_transform(predicted_test)
for i in range(0,len(predicted_test[0])):
    if(predicted_test[0][i]==1):
        print mapping_dict[i]
# print predicted_test

Enter text:I will kill you
anger
disgust
